In [9]:
import os

folder_pwd = os.getcwd()
os.chdir('..')
os.chdir('../resources')
resources_path = os.getcwd()
os.chdir(folder_pwd)

In [10]:
image_folder = "1"
csv_file = ["emery-labels-1-none.csv","emery-labels-1.csv","emery-labels-2.csv"]

# Read CSVs

In [11]:
%%time
from csv import reader
import pandas as pd
import numpy as np

def read_csv(file_name):
    return pd.read_csv(resources_path+'/images/'+file_name, names=['type','from_x','from_y','width', 'height', 'name', 'full_width', 'full_height'])

df = pd.DataFrame()
for f in csv_file:
    df = df.append(read_csv(f)).reset_index(drop=True)

cols = np.delete(df.columns.values,[0, 5])
df[cols] = df[cols].apply(pd.to_numeric)

df

Wall time: 11 ms


,type,from_x,from_y,width,height,name,full_width,full_height
0,none,338,221,79,90,19158_2021-01-08_20.27_202318153.png,800,480
1,none,236,240,102,113,19158_2021-01-08_20.27_202318153.png,800,480
2,none,498,386,74,19,19158_2021-01-08_20.27_202318153.png,800,480
3,none,294,278,72,79,19158_2021-01-08_20.27_202453308.png,800,480
4,none,371,117,181,60,19158_2021-01-08_20.27_202453308.png,800,480
...,...,...,...,...,...,...,...,...
965,head,241,169,174,214,20162_2021-02-17_21.40_21369317.png,800,480
966,head,398,165,166,157,20162_2021-02-19_13.47_134726794.png,800,480
967,spine,258,304,177,69,20162_2021-02-19_13.47_134726794.png,800,480
968,spine,371,149,95,66,20162_2021-02-21_22.17_22170208.png,800,480


# Small correction before extraction

In [12]:
%%time
def center_in_square(row):
    diff = row['width'] - row['height']
    if diff > 0:
        row['from_y'] -= int(abs(diff)/2)
        row['height'] = row['width']
    elif diff<0:
        row['from_x'] -= int(abs(diff)/2)
        row['width'] = row['height']
    return row

df = df.apply(center_in_square, axis=1)

neg1 = df['from_x']<0
neg2 = df['from_y']<0
to_remove = np.logical_or(neg1, neg2)
df = df[~to_remove]

df['to_x'] = df['from_x'] + df['width']
df['to_y'] = df['from_y'] + df['height']
df

Wall time: 52 ms


,type,from_x,from_y,width,height,name,full_width,full_height,to_x,to_y
0,none,333,221,90,90,19158_2021-01-08_20.27_202318153.png,800,480,423,311
1,none,231,240,113,113,19158_2021-01-08_20.27_202318153.png,800,480,344,353
2,none,498,359,74,74,19158_2021-01-08_20.27_202318153.png,800,480,572,433
3,none,291,278,79,79,19158_2021-01-08_20.27_202453308.png,800,480,370,357
4,none,371,57,181,181,19158_2021-01-08_20.27_202453308.png,800,480,552,238
...,...,...,...,...,...,...,...,...,...,...
965,head,221,169,214,214,20162_2021-02-17_21.40_21369317.png,800,480,435,383
966,head,398,161,166,166,20162_2021-02-19_13.47_134726794.png,800,480,564,327
967,spine,258,250,177,177,20162_2021-02-19_13.47_134726794.png,800,480,435,427
968,spine,371,135,95,95,20162_2021-02-21_22.17_22170208.png,800,480,466,230


# Extract subsections

In [35]:
%%time
import cv2

def extract_subsection(row):
    img = cv2.imread(resources_path+'/images/'+image_folder+'/'+row['name'], 0)
    subsect = img[row['from_y']:row['to_y'], row['from_x']:row['to_x']]
    # In case the crop was near an edge, creating the square could make it go outside image
    if subsect.shape[0] != subsect.shape[1]:
        diff = subsect.shape[0] - subsect.shape[1]
        if diff<0:
            subsect = np.pad(subsect, ((0, abs(diff)),(0,0)),constant_values=0)
        else:
            subsect = np.pad(subsect, ((0, 0),(0, abs(diff))),constant_values=0)
    return subsect

df['subsection'] = df.apply(extract_subsection, axis=1)
df

Wall time: 3.72 s


,type,from_x,from_y,width,height,name,full_width,full_height,to_x,to_y,subsection
0,none,333,221,90,90,19158_2021-01-08_20.27_202318153.png,800,480,423,311,"[[68, 68, 71, 67, 67, 67, 68, 72, 77, 86, 96, ..."
1,none,231,240,113,113,19158_2021-01-08_20.27_202318153.png,800,480,344,353,"[[124, 130, 132, 125, 124, 136, 150, 166, 184,..."
2,none,498,359,74,74,19158_2021-01-08_20.27_202318153.png,800,480,572,433,"[[141, 146, 149, 148, 144, 140, 137, 138, 142,..."
3,none,291,278,79,79,19158_2021-01-08_20.27_202453308.png,800,480,370,357,"[[181, 189, 200, 204, 203, 209, 216, 223, 226,..."
4,none,371,57,181,181,19158_2021-01-08_20.27_202453308.png,800,480,552,238,"[[104, 125, 133, 111, 103, 111, 139, 151, 139,..."
...,...,...,...,...,...,...,...,...,...,...,...
965,head,221,169,214,214,20162_2021-02-17_21.40_21369317.png,800,480,435,383,"[[54, 56, 57, 49, 44, 40, 41, 43, 47, 54, 61, ..."
966,head,398,161,166,166,20162_2021-02-19_13.47_134726794.png,800,480,564,327,"[[91, 96, 98, 109, 120, 121, 120, 117, 115, 12..."
967,spine,258,250,177,177,20162_2021-02-19_13.47_134726794.png,800,480,435,427,"[[40, 35, 27, 22, 21, 22, 28, 26, 20, 12, 10, ..."
968,spine,371,135,95,95,20162_2021-02-21_22.17_22170208.png,800,480,466,230,"[[44, 45, 48, 43, 37, 32, 31, 28, 33, 37, 34, ..."


# Save image to appropriate folder

In [36]:
%%time
from matplotlib import pyplot
from pathlib import Path


def save_subsection(row):
    path = resources_path+'/features/'+row['type']
    Path(path).mkdir(parents=True, exist_ok=True)
    pyplot.imsave(path+'/'+row['name'], row['subsection'])

df.apply(save_subsection, axis=1)

Wall time: 8.56 s


0      None
1      None
2      None
3      None
4      None
       ... 
965    None
966    None
967    None
968    None
969    None
Length: 962, dtype: object